## Import data

In [1]:
from brightway2 import *
from bw2data import Database, get_activity
import numpy as np
import pandas as pd


In [2]:
projects.set_current('Import Quantis OFP - version 4') 

In [3]:
bw2setup()

Biosphere database already present!!! No setup is needed


In [4]:
list(databases)

['MeatDatasets', 'AGs', 'ecoinvent 3.3 cutoff', 'biosphere3']

# calculate LCIA

try first sth from ecoinvent

In [6]:
act = Database("MeatDatasets").search('suckler average')[1]
act


'Suckler beef average, conventional (Quantis, from Agribalyse) UP/DE' (kilogram, None, None)

In [7]:
lca = LCA(
    {act.key: 1}, 
    method=('IPCC 2013', 'climate change', 'GWP 100a'),
)
lca.lci()
lca.lcia()
lca.score

18.4052599249694

In [74]:
# help(lca.characterized_inventory)
# type(lca.characterized_inventory)
lca.characterized_inventory.shape

(2156, 14236)

## Contribution Analysis

In [8]:
from bw2analyzer import ContributionAnalysis

In [9]:
results = ContributionAnalysis().annotated_top_processes(lca, names=False)
results = [(x[0], x[1], get_activity(x[2]), x[2]) for x in results]
df = pd.DataFrame(results, columns=['lca_score', 'inventory_amount', 'activity', 'process_key'])
# print(df.activity.apply(type)) # peewee objects...
# print(df.activity.apply(len))
# For example
df.activity


0     [code, filename, type, production amount, sima...
1     [code, filename, type, production amount, sima...
2     [code, filename, type, production amount, sima...
3     [code, filename, type, production amount, sima...
4     [code, filename, type, production amount, sima...
5     [code, filename, type, production amount, sima...
6     [classifications, filename, type, production a...
7     [code, filename, type, production amount, sima...
8     [code, filename, type, production amount, sima...
9     [code, filename, type, production amount, sima...
10    [code, filename, type, production amount, sima...
11    [classifications, filename, type, production a...
12    [code, filename, type, production amount, sima...
13    [classifications, filename, type, production a...
14    [code, filename, type, production amount, sima...
15    [classifications, filename, type, production a...
16    [classifications, filename, type, production a...
17    [classifications, filename, type, producti

In [10]:
df = lca.to_dataframe()
df.head()


                                            Activity  \
0  Young suckler bull, conventional, fattening sy...   
1  Young suckler bull, conventional, fattening sy...   
2                            Compost, at plant UP/FR   
3                            Compost, at plant UP/FR   
4                            Compost, at plant UP/FR   

                          Flow Region    Amount  
0  Carbon monoxide, non-fossil   None  3.151814  
1  Carbon monoxide, non-fossil   None  2.527043  
2  Carbon monoxide, non-fossil   None  2.139878  
3  Carbon monoxide, non-fossil   None  1.269280  
4  Carbon monoxide, non-fossil   None  0.985291  

In [11]:
lca.to_dataframe

<bound method LCA.to_dataframe of <bw2calc.lca.LCA object at 0x7fc5f76dcf98>>

In [85]:
lca.characterized_inventory

<2156x14236 sparse matrix of type '<class 'numpy.float64'>'
	with 15695 stored elements in Compressed Sparse Row format>

In [34]:
df.sort_values(by='Amount', ascending=False, inplace=True)


In [35]:
lca.top_emissions()

[(11.653903594071792,
  0.4089088980376071,
  'Methane, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (3.3550709038030657,
  0.012670207924730675,
  'Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (1.5937783154278489,
  1.5937783154278478,
  'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (0.80535814152737173,
  0.80535814152737206,
  'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.38303347947197897,
  0.38303347947197863,
  'Carbon dioxide, fossil' (kilogram, None, ('air',))),
 (0.32941719902933975,
  0.0012440227122929152,
  'Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.14777017123808658,
  0.0049754265132032809,
  'Methane, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (0.065395837196283488,
  0.065395837196283474,
  'Carbon dioxide, from soil or biomass stock' (kilogr

In [39]:
c_an = ContributionAnalysis().annotated_top_emissions(lca, {'limit': 0, 'limit_type':'percent', 'total':None})
pd.DataFrame(c_an)


NameError: name 'pd' is not defined

## Create report

In [0]:
from bw2analyzer.report import SerializedLCAReport
test = SerializedLCAReport(
    activity={act.key: 1},
    method=('IPCC 2013', 'climate change', 'GWP 100a'),
    iterations=10,
)

NameError: name 'act' is not defined

In [85]:
test.calculate()

DatabaseError: database disk image is malformed

FD
CA
hinton
treemap
herfindahl
concentration
MC:
Entered get_monte_carlo


In [0]:
test.report['monte carlo']['histogram']= list(test.report['monte carlo']['histogram'])

In [0]:
test.report['monte carlo']['smoothed'] = list(test.report['monte carlo']['smoothed'])

In [0]:
test.write()

In [0]:
test.report

In [0]:
sorted(str(get_activity(key)) for key, _ in Method(('IPCC 2013', 'climate change', 'GWP 100a')).load())

See all flows with their characterization factors

In [4]:
from brightway2 import *
# method = methods.random()
methods = Method(('IPCC 2013', 'climate change', 'GWP 100a')).load()
for key, cf in methods:
    print(get_activity(key), cf)

UnknownObject: This object is not registered and has no data